# loading modules

In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from model import scVI_final as scVI
from benchmarking import *
from helper import *

# parameters

In [2]:
run_batch_scVI = True
learning_rate = 0.0004
epsilon = 0.01

# import data

In [3]:
# expression data
data_path = "/home/ubuntu/single-cell-scVI/data/10xPBMCs/"
expression_train = np.load(data_path + "de/data_train.npy")
expression_test = np.load(data_path + "de/data_test.npy")

# qc metrics
r_train = np.load(data_path + "design_train.npy")
r_test = np.load(data_path + "design_test.npy")

# labels
c_train = np.loadtxt(data_path + "label_train")
c_test = np.loadtxt(data_path + "label_test")

# batch info
b_train = np.loadtxt(data_path + "b_train")
b_test = np.loadtxt(data_path + "b_test")

# corrupted data
X_zero, i, j, ix = \
        np.load(data_path + "imputation/X_zero.npy"), np.load(data_path + "imputation/i.npy"),\
        np.load(data_path + "imputation/j.npy"), np.load(data_path + "imputation/ix.npy")

# Computational graph

In [5]:
tf.reset_default_graph()
expression = tf.placeholder(tf.float32, (None, expression_train.shape[1]), name='x')
kl_scalar = tf.placeholder(tf.float32, (), name='kl_scalar')
batch = tf.placeholder(tf.int32, [None], name="batch_ind")
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=epsilon)
training_phase = tf.placeholder(tf.bool, (), name='training_phase')
mmd_scalar = tf.placeholder(tf.float32, [], name='l')

# setting up priors
l_mean, l_var = [], []
log_library_size = np.log(np.sum(expression_train, axis=1))
for i in np.unique(b_train):
    l_mean.append([np.mean(log_library_size[b_train == i])])
    l_var.append([np.var(log_library_size[b_train == i])])
library_mean = tf.constant(l_mean)
library_var = tf.constant(l_var)

# loading model
if not run_batch_scVI:
    mean = np.mean(l_mean)
    var = np.mean(l_var)

    model = scVI.scVIModel(expression=expression, kl_scale=kl_scalar, \
                          optimize_algo=optimizer, phase=training_phase, \
                           library_size_mean=mean, library_size_var=var)
    
else:
    model = scVI.scVIModel(expression=expression, kl_scale=kl_scalar, batch_ind=batch, mmd_scale=mmd_scalar, \
                       num_batches=2, apply_mmd=True, optimize_algo=optimizer, phase=training_phase, \
                           dispersion="gene-batch", library_size_mean=library_mean, library_size_var=library_var)
    
# Session creation
sess = tf.Session()

Running scVI on 3346 genes
Got 2batches in the data
Will apply a MMD penalty
Will work on mode list for incorporating library size
Will work on mode gene-batch for modeling inverse dispersion param
Will apply zero inflation
1 hidden layers at 128 each for a final 10 latent space


In [6]:
sess.run(tf.global_variables_initializer())
result = train_model(model, (expression_train, expression_test), sess, 120, batch=(b_train, b_test))

In [47]:
expression_train.shape, expression_test.shape

((9029, 3346), (3010, 3346))

In [4]:
from sklearn.decomposition import FactorAnalysis

In [43]:
def sample_generative_model(N):
    z_train = np.random.normal(size=(N, 10))
    l_train = np.mean(l_mean) + np.sqrt(np.mean(l_var)) * np.random.normal(size=(N, 1))
    b_train = np.ones((N))
    dic_z = {model.z: z_train, model.library:l_train, training_phase:False, kl_scalar:1., batch:b_train}
    rate, dropout = sess.run((model.px_rate, model.px_dropout), feed_dict=dic_z)
    dispersion = np.tile(sess.run((tf.exp(model.px_r))[0]), (rate.shape[0], 1))
    p = rate / (rate + dispersion)
    r = dispersion 
    dropout = 1. / (1. + np.exp(-dropout))
    l_train = np.random.gamma(r, p / (1-p))
    X_train = np.random.poisson(l_train)
    X_train *= np.random.binomial(1, 1-dropout)
    return X_train

In [50]:
eval_likelihood(model, sample_generative_model(3010), sess, batch=np.ones((3010)))

1490.6595

In [10]:
fa = FactorAnalysis(n_components=10, noise_variance_init=10000 * np.ones(expression_train.shape[1]))
fa.fit(np.log(1 + expression_train))
test_set = np.log(1 + expression_test)
fa.score(test_set) - np.mean(np.sum(test_set, axis=-1))

359.9556228982517

In [49]:
fa = FactorAnalysis(n_components=10)
fa.fit(np.log(1 + sample_generative_model(9029)))
test_set = np.log(1 + sample_generative_model(3010))
fa.score(test_set) - np.mean(np.sum(test_set, axis=-1))

1176.7996554912788

In [51]:
latent = eval_latent(model, expression_train, sess, batch=b_train)
cluster_scores(latent, len(np.unique(c_train)), c_train)

[0.38777304, 0.79069378213836394, 0.69595041584939288]

# Evaluation methods

In [53]:
res = train_model(model, (expression_test, expression_test), sess, 100, \
                  step=model.test_step, batch=(b_test, b_test))
eval_likelihood(model, expression_test, sess, batch=b_test)

1361.0693

### imputation

In [63]:
sess.run(tf.global_variables_initializer())
res = train_model(model, (expression_train, expression_test), sess, 120, batch=(b_train, b_test))
eval_imputed_data(model, (X_zero, i, j, ix), expression_train, sess, batch=b_train)

(9029, 3346)


0.090907849371433258